In [11]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Polygon
import json
import re
import geojson
from geojson import MultiLineString
from folium.plugins import HeatMap

In [1]:
import pandas as pd
import numpy as np
incidents = pd.read_csv('Traffic_Incidents.csv')
incidents = incidents.rename(columns = {'Latitude' : 'latitude', 'Longitude':'longitude'})
incidents = incidents[[ "latitude","longitude",]]
incidents['longitude'].sort_values()
incidents['latitude'].sort_values()

signals = pd.read_csv('Traffic_Signals.csv')
signals = signals[["latitude", 'longitude']]
signals

cameras = pd.read_csv('Traffic_Camera_Locations.csv')
cameras = cameras[['latitude', 'longitude']]
cameras

speed = pd.read_csv('Speed_Limits.csv')
speed['multiline'] = speed['multiline'].map(lambda x: x.lstrip('MULTILINESTRING (('))
speed['multiline'] = speed['multiline'].str.split(',').str[0]
speed['longitude'] = speed['multiline'].str.split(' ').str[0]
speed['latitude'] = speed['multiline'].str.split(' ').str[1]
speed = speed[['latitude', 'longitude', 'SPEED']]
speed = speed.apply(pd.to_numeric)

volume = pd.read_csv('Traffic_Volumes_for_2018.csv')
volume['multilinestring'] = volume['multilinestring'].map(lambda x: x.lstrip('MULTILINESTRING (('))
volume['multilinestring'] = volume['multilinestring'].str.split(',').str[0]
volume['longitude'] = volume['multilinestring'].str.split(' ').str[0]
volume['latitude'] = volume['multilinestring'].str.split(' ').str[1]
volume = volume[['VOLUME', 'latitude', 'longitude']]
volume = volume.apply(pd.to_numeric)

display(incidents, signals, cameras, speed, volume)

,latitude,longitude
0,51.097316,-114.083318
1,51.070539,-114.081378
2,51.067299,-113.984934
3,51.152736,-114.147933
4,51.119684,-114.203241
...,...,...
19783,50.946741,-114.005304
19784,50.912974,-114.069753
19785,51.152731,-114.150728
19786,51.183220,-114.134978


,latitude,longitude
0,51.052514,-114.091740
1,51.154211,-114.052475
2,51.094081,-114.011478
3,51.117992,-114.069588
4,51.037739,-114.025122
...,...,...
1533,51.154147,-114.228956
1534,50.902069,-114.109199
1535,51.073213,-114.084589
1536,51.048341,-114.060387


,latitude,longitude
0,50.900726,-113.976606
1,51.053253,-113.955818
2,51.098849,-114.149379
3,51.111255,-114.178204
4,51.139352,-114.001451
...,...,...
121,51.050802,-114.050136
122,50.979446,-113.981495
123,50.979635,-113.929263
124,51.048677,-114.073644


,latitude,longitude,SPEED
0,50.913577,-114.073658,60
1,50.996224,-114.034764,40
2,51.166714,-114.068566,60
3,51.005002,-114.117825,70
4,51.062448,-113.918851,80
...,...,...,...
1447,51.091535,-114.233242,80
1448,50.917759,-114.140310,70
1449,50.904313,-114.030210,60
1450,51.124175,-114.177871,60


,VOLUME,latitude,longitude
0,22000,51.048319,-114.060367
1,5000,50.968634,-114.068768
2,5000,51.053232,-114.033722
3,2000,51.143370,-114.013188
4,14000,50.957841,-113.981288
...,...,...,...
1764,44000,51.154205,-114.027976
1765,27000,50.972345,-113.981983
1766,20000,51.065390,-114.013382
1767,7000,50.875745,-114.035671


In [2]:
startlong = -114.31
endlong = -113.85
startlat = 50.84
endlat = 51.21
lowerLeft = [startlat, startlong]
upperRight = [endlat, endlong]

def get_geojson_grid(upper_right, lower_left, n, m):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.
    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).
    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).
    n: integer
        The number of rows/columns in the (n,n) grid.
    list
        List of "geojson style" dictionary objects   
    """
    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], m+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

        
            geo_json = {
                        "lower_left": lower_left,
                        "lower_right": lower_right,
                            "upper_left": upper_left,
                            "upper_right": upper_right
                        }

          
            all_boxes.append(geo_json)

    return all_boxes

gridLines = get_geojson_grid(upperRight, lowerLeft,10,10)
df = pd.DataFrame(data = gridLines)
df = df.astype(str)


df['minX'] = df["lower_left"].str.split(',').str[1]
df['minX'] = df['minX'].str.split(']').str[0]

df['maxX'] = df['upper_right'].str.split(',').str[1]
df['maxX'] = df['maxX'].str.split(']').str[0]

df['minY'] = df['lower_left'].str.split(',').str[0]
df['minY'] = df['minY'].str.split('[').str[1]

df['maxY'] = df['upper_right'].str.split('[').str[1]
df['maxY'] = df['maxY'].str.split(',').str[0]
df

#df.to_csv('file_name123.csv', index = False)

                    
df = df[['minX','maxX', 'minY', 'maxY']]
df['minX'] = df['minX'].map(lambda x: x.lstrip(' '))
df['maxX'] = df['maxX'].map(lambda x: x.lstrip(' '))
df = df.apply(pd.to_numeric)
#df['minX'] = df['minX'].astype('int64')
df1 = df.head(100)
df

,minX,maxX,minY,maxY
0,50.840,50.877,-114.310,-114.264
1,50.840,50.877,-114.264,-114.218
2,50.840,50.877,-114.218,-114.172
3,50.840,50.877,-114.172,-114.126
4,50.840,50.877,-114.126,-114.080
...,...,...,...,...
95,51.173,51.210,-114.080,-114.034
96,51.173,51.210,-114.034,-113.988
97,51.173,51.210,-113.988,-113.942
98,51.173,51.210,-113.942,-113.896


In [3]:
gridLines

[{'lower_left': [-114.31, 50.84],
  'lower_right': [-114.264, 50.84],
  'upper_left': [-114.31, 50.877],
  'upper_right': [-114.264, 50.877]},
 {'lower_left': [-114.264, 50.84],
  'lower_right': [-114.21799999999999, 50.84],
  'upper_left': [-114.264, 50.877],
  'upper_right': [-114.21799999999999, 50.877]},
 {'lower_left': [-114.218, 50.84],
  'lower_right': [-114.172, 50.84],
  'upper_left': [-114.218, 50.877],
  'upper_right': [-114.172, 50.877]},
 {'lower_left': [-114.172, 50.84],
  'lower_right': [-114.12599999999999, 50.84],
  'upper_left': [-114.172, 50.877],
  'upper_right': [-114.12599999999999, 50.877]},
 {'lower_left': [-114.126, 50.84],
  'lower_right': [-114.08, 50.84],
  'upper_left': [-114.126, 50.877],
  'upper_right': [-114.08, 50.877]},
 {'lower_left': [-114.08, 50.84],
  'lower_right': [-114.03399999999999, 50.84],
  'upper_left': [-114.08, 50.877],
  'upper_right': [-114.03399999999999, 50.877]},
 {'lower_left': [-114.03399999999999, 50.84],
  'lower_right': [-113.9

In [66]:
def checkPoint(dataFrame, series, name):
    """checkPoint takes in a list of tuple argument called points and it will pass each x,y coordinate in the list and if the point is located inside the grid specifications, the count increases.
    """
    df = dataFrame
    s = series
    county = []
    count = 0
    for index, row in df.iterrows():
        for indexy, seriesrows in s.iterrows():
            if (seriesrows['latitude'] >= row['minX'] and seriesrows['latitude'] < row['maxX']):
                if(seriesrows['longitude'] >= row['minY'] and seriesrows['longitude'] < row['maxY']):
                    count += 1
                    s.drop(indexy, inplace = True)
                    
        county.append(count)
        count = 0
    
    df[name] = county
    return df
        
df = checkPoint(df,incidents,"accident")
df

,minX,maxX,minY,maxY,accident
0,50.840,50.877,-114.310,-114.264,0
1,50.840,50.877,-114.264,-114.218,0
2,50.840,50.877,-114.218,-114.172,0
3,50.840,50.877,-114.172,-114.126,0
4,50.840,50.877,-114.126,-114.080,0
...,...,...,...,...,...
95,51.173,51.210,-114.080,-114.034,36
96,51.173,51.210,-114.034,-113.988,105
97,51.173,51.210,-113.988,-113.942,65
98,51.173,51.210,-113.942,-113.896,3


In [67]:
df['accident'].sort_values()
df = checkPoint(df,signals,"signals")
df

,minX,maxX,minY,maxY,accident,signals
0,50.840,50.877,-114.310,-114.264,0,0
1,50.840,50.877,-114.264,-114.218,0,0
2,50.840,50.877,-114.218,-114.172,0,0
3,50.840,50.877,-114.172,-114.126,0,0
4,50.840,50.877,-114.126,-114.080,0,0
...,...,...,...,...,...,...
95,51.173,51.210,-114.080,-114.034,36,1
96,51.173,51.210,-114.034,-113.988,105,0
97,51.173,51.210,-113.988,-113.942,65,0
98,51.173,51.210,-113.942,-113.896,3,0


In [68]:
#startlong = -114.25
#long1 = startlong + 0.09
#long2 = long1 + 0.09
#long3 = long2 + 0.09
#endlong = -113.89

#startlat = 50.85
#lat1 = startlat + 0.06
#lat2 = lat1 + 0.06
#lat3 = lat2 + 0.06
#lat4 = lat3 + 0.06
#lat5 = lat4 + 0.06
#endlat = 51.21

In [69]:
df = checkPoint(df,cameras,"cameras")
df

,minX,maxX,minY,maxY,accident,signals,cameras
0,50.840,50.877,-114.310,-114.264,0,0,0
1,50.840,50.877,-114.264,-114.218,0,0,0
2,50.840,50.877,-114.218,-114.172,0,0,0
3,50.840,50.877,-114.172,-114.126,0,0,0
4,50.840,50.877,-114.126,-114.080,0,0,0
...,...,...,...,...,...,...,...
95,51.173,51.210,-114.080,-114.034,36,1,0
96,51.173,51.210,-114.034,-113.988,105,0,1
97,51.173,51.210,-113.988,-113.942,65,0,0
98,51.173,51.210,-113.942,-113.896,3,0,0


In [70]:
def checkPointAverage(dataFrame, series, name, parameter):
    """checkPoint takes in a list of tuple argument called points and it will pass each x,y coordinate in the list and if the point is located inside the grid specifications, the count increases.
    """
    df = dataFrame
    s = series
    array = []
    county = []
    sum = 0
    average = 0
    count = 0
    for index, row in df.iterrows():
        for indexy, seriesrows in s.iterrows():
            if (seriesrows['latitude'] >= row['minX'] and seriesrows['latitude'] < row['maxX']):
                if(seriesrows['longitude'] >= row['minY'] and seriesrows['longitude'] < row['maxY']):
                    count += 1
                    sum += seriesrows[parameter]
                    s.drop(indexy, inplace = True)
        if count == 0:
            county.append(count)
            array.append(count)
        else:
            array.append(sum/count)            
            county.append(count)
        count = 0
        sum = 0
        average = 0
    df[parameter] = array
    df[name] = county
    return df


,minX,maxX,minY,maxY,accident,signals,cameras
0,50.840,50.877,-114.310,-114.264,0,0,0
1,50.840,50.877,-114.264,-114.218,0,0,0
2,50.840,50.877,-114.218,-114.172,0,0,0
3,50.840,50.877,-114.172,-114.126,0,0,0
4,50.840,50.877,-114.126,-114.080,0,0,0
...,...,...,...,...,...,...,...
95,51.173,51.210,-114.080,-114.034,36,1,0
96,51.173,51.210,-114.034,-113.988,105,0,1
97,51.173,51.210,-113.988,-113.942,65,0,0
98,51.173,51.210,-113.942,-113.896,3,0,0


In [71]:

df = checkPointAverage(df, speed, "Average Speed Count", "SPEED")
df

,minX,maxX,minY,maxY,accident,signals,cameras,SPEED,Average Speed Count
0,50.840,50.877,-114.310,-114.264,0,0,0,0.00,0
1,50.840,50.877,-114.264,-114.218,0,0,0,0.00,0
2,50.840,50.877,-114.218,-114.172,0,0,0,0.00,0
3,50.840,50.877,-114.172,-114.126,0,0,0,0.00,0
4,50.840,50.877,-114.126,-114.080,0,0,0,0.00,0
...,...,...,...,...,...,...,...,...,...
95,51.173,51.210,-114.080,-114.034,36,1,0,68.75,8
96,51.173,51.210,-114.034,-113.988,105,0,1,76.50,20
97,51.173,51.210,-113.988,-113.942,65,0,0,63.75,16
98,51.173,51.210,-113.942,-113.896,3,0,0,100.00,3


In [72]:
df = checkPointAverage(df, volume, "Average Volume Count", "VOLUME")
df

,minX,maxX,minY,maxY,accident,signals,cameras,SPEED,Average Speed Count,VOLUME,Average Volume Count
0,50.840,50.877,-114.310,-114.264,0,0,0,0.00,0,0.0,0
1,50.840,50.877,-114.264,-114.218,0,0,0,0.00,0,0.0,0
2,50.840,50.877,-114.218,-114.172,0,0,0,0.00,0,0.0,0
3,50.840,50.877,-114.172,-114.126,0,0,0,0.00,0,0.0,0
4,50.840,50.877,-114.126,-114.080,0,0,0,0.00,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
95,51.173,51.210,-114.080,-114.034,36,1,0,68.75,8,17200.0,5
96,51.173,51.210,-114.034,-113.988,105,0,1,76.50,20,77000.0,2
97,51.173,51.210,-113.988,-113.942,65,0,0,63.75,16,27500.0,2
98,51.173,51.210,-113.942,-113.896,3,0,0,100.00,3,0.0,0


In [74]:
#Checking if all the data points were used
display(volume,incidents, cameras, signals,speed )

,VOLUME,latitude,longitude


,latitude,longitude
10719,0.00511,-114.039384
16292,0.00511,-114.039384


,latitude,longitude


,latitude,longitude


,latitude,longitude,SPEED


In [14]:
volume.dtypes

VOLUME         int64
latitude     float64
longitude    float64
dtype: object

In [15]:
heatmap=[]
for index, row in volume.iterrows():
    heatmap.append([row['latitude'], row['longitude'], row['VOLUME']])
    

In [31]:
heatmap[:5]

[[51.04831941917631, -114.06036700906715, 22000.0],
 [50.968634255733654, -114.06876825342, 5000.0],
 [51.053232488239935, -114.03372206187294, 5000.0],
 [51.143369698984344, -114.01318801277269, 2000.0],
 [50.957841141776285, -113.98128756128924, 14000.0]]

In [28]:
m = folium.Map(location=[51.044270, -114.062019], zoom_start=12)

In [32]:
HeatMap(data = heatmap[:5], radius = 10).add_to(m)
m

In [ ]:
[y.split() for y in x]